In [1]:
import os
import json
import math
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
class Evaluation_nDCG:
    def __init__(self, database_file):
        with open(database_file, 'r') as f:
            self.database = json.loads(f.read())
            self.fps = dict()
            for data in self.database:
                self.fps[data['file_index']] = data['screenshots']['fps']
        return;
    
    def frame_index(self, fps, time):
        return int(fps * time)

    def to_milliseconds(self, time_str):
        time_tokens = time_str.split(':')
        mSec = int(time_tokens[0])*3600 + int(time_tokens[1]) * 60 + int(time_tokens[2])
        return mSec

    def valid_answer(self, file_id, frame_id, answer_list):
        hit = False
        for answer in answer_list:
            video_id = answer['video_id']
            frame_start = self.frame_index(self.fps[file_id], self.to_milliseconds(answer['timeslot'][0]))
            frame_stop = self.frame_index(self.fps[file_id], self.to_milliseconds(answer['timeslot'][1]))
            #print(f'results:(file_id={file_id}, frame_id={frame_id}),answers:(video_id={video_id},frame_range=({frame_start}, {frame_stop}) )')
            if file_id == video_id and (int(frame_id) >= frame_start and int(frame_id) <= frame_stop):
                hit = True
        return hit
    
    #Discounted Cumulative Gain 
    def calc_nDCG(self, answer_data, rst_data, rank_pos):
        DCG = 0
        IDCG = 0
        nDCG = 0
        rst_dir = {}
        for n in range(len(rst_data)):
            data = rst_data[n]
            file_id = data['file']
            frame_id = data['frame_id']
            if self.valid_answer(file_id, frame_id, answer_data):
                print(f'******Hit! Found an answer at rank {n+1}!******')
                rst_dir[n+1] = 1 #The relevancy will always be binary 0 or 1
            else:
                rst_dir[n+1] = 0
       
        #calculate DCG 
        DCG = sum([rel/math.log2(rank + 1) for rank, rel in rst_dir.items()][:rank_pos])
        #print(f"DCG={DCG}")
        #Calculate IDCG
        order_lst = sorted(rst_dir.values(), reverse=True)
        IDCG = sum([rel/math.log2(rank + 2) for rank, rel in enumerate(order_lst)][:rank_pos])
        #print(f"IDCG={IDCG}")
        
        if IDCG != 0:
            nDCG = DCG/IDCG
        return nDCG

    def evaluation_mean_nDCG(self, result_file, answer_file, rank_pos=10): 
        json_file = open(result_file)
        result_data = json.loads(json_file.read())
        json_file = open(answer_file)
        answer_data = json.loads(json_file.read())
        
        #print("number of results:", len(result_data.keys()))
        sum_nDCG = 0
        for query, results in result_data.items():
            print(f'query: {query}')
            if query in answer_data.keys():
                answer_list = answer_data[query]
                sum_nDCG += self.calc_nDCG(answer_list, results, rank_pos)
        #print(sum_nDCG)
        mean_nDCG = sum_nDCG/len(result_data.keys())

        return mean_nDCG

In [3]:
database_file = './data source/Life Is Strange 1/output 1/Life Is Strange.json'
answer_file = './data source/Life Is Strange 1/output 1/correct_answers.json'
result_file = './data source/Life Is Strange 1/output 1/Life Is Strange_results.json'
tester0 = Evaluation_nDCG(database_file)
tester0.evaluation_mean_nDCG(result_file, answer_file)

query: deduction scene
query: rewind prompt
query: Max fire alarm Chloe Rescue
******Hit! Found an answer at rank 9!******
query: Butterfly Bucket
******Hit! Found an answer at rank 1!******
query: Nathan Gun Chloe
query: danger gun
query: Time Freeze Kate Birds
query: Kate Rooftop Max
query: storm lightening
******Hit! Found an answer at rank 3!******
******Hit! Found an answer at rank 4!******
query: storm cliff tornado
query: toilet gun
query: selfie classroom
******Hit! Found an answer at rank 6!******
******Hit! Found an answer at rank 7!******
******Hit! Found an answer at rank 8!******
query: Max gallery
query: Kate hospital
query: Restaurant Jukebox
******Hit! Found an answer at rank 3!******
query: swimming pool
******Hit! Found an answer at rank 7!******
******Hit! Found an answer at rank 8!******
query: railway train stuck
query: beacon
query: sea sunset
******Hit! Found an answer at rank 9!******
query: jukebox cafe
******Hit! Found an answer at rank 3!******
******Hit! Fou

0.23176734796586682

In [5]:
database_file = './data source/Life Is Strange 1/output 1/lifeisstrange_w_img_top5keywords.json'
answer_file = './data source/Life Is Strange 1/output 1/correct_answers.json'
result_file = './data source/Life Is Strange 1/output 1/lifeisstrange_w_img_top5keywords_results.json'
tester1 = Evaluation_nDCG(database_file)
tester1.evaluation_mean_nDCG(result_file, answer_file)

query: deduction scene
query: rewind prompt
query: Max fire alarm Chloe Rescue
query: Butterfly Bucket
******Hit! Found an answer at rank 4!******
query: Nathan Gun Chloe
query: danger gun
******Hit! Found an answer at rank 7!******
******Hit! Found an answer at rank 10!******
query: Time Freeze Kate Birds
query: Kate Rooftop Max
query: storm lightening
******Hit! Found an answer at rank 7!******
query: storm cliff tornado
******Hit! Found an answer at rank 1!******
******Hit! Found an answer at rank 2!******
******Hit! Found an answer at rank 3!******
******Hit! Found an answer at rank 5!******
******Hit! Found an answer at rank 7!******
query: toilet gun
query: selfie classroom
******Hit! Found an answer at rank 1!******
******Hit! Found an answer at rank 2!******
******Hit! Found an answer at rank 3!******
******Hit! Found an answer at rank 8!******
******Hit! Found an answer at rank 10!******
query: Max gallery
query: Kate hospital
query: Restaurant Jukebox
query: swimming pool
***

0.3220151447517286

In [6]:
database_file = './data source/Life Is Strange 1/output 1/lifeisstrange_w_img_top3keywords.json'
answer_file = './data source/Life Is Strange 1/output 1/correct_answers.json'
result_file = './data source/Life Is Strange 1/output 1/lifeisstrange_w_img_top3keywords_results.json'
tester2 = Evaluation_nDCG(database_file)
tester2.evaluation_mean_nDCG(result_file, answer_file)

query: deduction scene
query: rewind prompt
query: Max fire alarm Chloe Rescue
query: Butterfly Bucket
******Hit! Found an answer at rank 1!******
query: Nathan Gun Chloe
query: danger gun
******Hit! Found an answer at rank 1!******
******Hit! Found an answer at rank 4!******
******Hit! Found an answer at rank 8!******
query: Time Freeze Kate Birds
query: Kate Rooftop Max
query: storm lightening
query: storm cliff tornado
******Hit! Found an answer at rank 1!******
******Hit! Found an answer at rank 3!******
******Hit! Found an answer at rank 8!******
query: toilet gun
query: selfie classroom
******Hit! Found an answer at rank 1!******
******Hit! Found an answer at rank 2!******
******Hit! Found an answer at rank 9!******
******Hit! Found an answer at rank 10!******
query: Max gallery
******Hit! Found an answer at rank 5!******
query: Kate hospital
query: Restaurant Jukebox
query: swimming pool
******Hit! Found an answer at rank 10!******
query: railway train stuck
query: beacon
******

0.33795350781796013

In [7]:
database_file = './data source/Life Is Strange 1/output 1/lifeisstrange_w_img_top3keywords.json'
answer_file = './data source/Life Is Strange 1/output 1/correct_answers.json'
result_file = './data source/Life Is Strange 1/output 1/lifeisstrange_w_img_top3keywords_results_ma.json' # window_size 5
tester3 = Evaluation_nDCG(database_file)
tester3.evaluation_mean_nDCG(result_file, answer_file)

query: deduction scene
query: rewind prompt
query: Max fire alarm Chloe Rescue
query: Butterfly Bucket
******Hit! Found an answer at rank 3!******
******Hit! Found an answer at rank 9!******
query: Nathan Gun Chloe
query: danger gun
******Hit! Found an answer at rank 1!******
******Hit! Found an answer at rank 3!******
******Hit! Found an answer at rank 4!******
query: Time Freeze Kate Birds
query: Kate Rooftop Max
query: storm lightening
******Hit! Found an answer at rank 10!******
query: storm cliff tornado
******Hit! Found an answer at rank 1!******
******Hit! Found an answer at rank 2!******
******Hit! Found an answer at rank 4!******
******Hit! Found an answer at rank 6!******
******Hit! Found an answer at rank 9!******
******Hit! Found an answer at rank 10!******
query: toilet gun
query: selfie classroom
******Hit! Found an answer at rank 1!******
******Hit! Found an answer at rank 3!******
******Hit! Found an answer at rank 7!******
query: Max gallery
query: Kate hospital
query:

0.3150363891404884

In [8]:
database_file = './data source/Life Is Strange 1/output 1/lifeisstrange_w_img_top3keywords.json'
answer_file = './data source/Life Is Strange 1/output 1/correct_answers.json'
result_file = './data source/Life Is Strange 1/output 1/lifeisstrange_w_img_top3keywords_results_ma10.json' # window_size 10
tester3 = Evaluation_nDCG(database_file)
tester3.evaluation_mean_nDCG(result_file, answer_file)

query: storm cliff tornado
******Hit! Found an answer at rank 1!******
******Hit! Found an answer at rank 2!******
******Hit! Found an answer at rank 3!******
******Hit! Found an answer at rank 4!******
******Hit! Found an answer at rank 5!******
******Hit! Found an answer at rank 6!******
******Hit! Found an answer at rank 7!******
******Hit! Found an answer at rank 8!******
******Hit! Found an answer at rank 9!******


1.0